In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.utils.data as data
import numpy as np
import datetime
import random

data_points = []
item_list = { # Mapping of item_id to price
    1001: 2,
    1002: 4,
    1003: 10,
    1004: 20,
    1005: 40,
    1006: 60
}
months = {
    0: 0.95,
    1: 0.25,
    2: 0.22,
    3: 0.20,
    4: 0.23,
    5: 0.24,
    6: 0.29,
    7: 0.55,
    8: 0.36,
    9: 0.25,
    10: 0.21,
    11: 0.49
}
timespan = datetime.datetime.now() - datetime.timedelta(hours=96)
with open("locations.txt") as f:
    lines = f.readlines()
    for i in range(len(lines)):
        random_val = random.random()
        second_random_val = random.random()
        lat, lon = eval(lines[i])
        for key, value in item_list.items():
            now = datetime.datetime.now()
            for weeks_ago in range(0, 260):
                data_points.append((i, key, weeks_ago, months[(weeks_ago // 4) % 12], value + i + months[(weeks_ago // 4) % 12]))
    

In [26]:
a = torch.tensor([1, 2, 3, 4])
print(a.unsqueeze(0))

tensor([[1, 2, 3, 4]])


In [7]:
def create_dataset(dataset, lookback):
    """Transform a time series into a prediction dataset
    
    Args:
        dataset: A numpy array of time series, first dimension is the time steps
        lookback: Size of window for prediction
    """
    X = []
    for i in range(len(dataset)-lookback):
        feature = dataset[i:i+lookback]
        X.append(feature)
    return torch.tensor(X, dtype=torch.float)

lookback = 2
test_size = 250
standardized_data_points = []
df_p = np.array([data[-1] for data in data_points])
#, columns=["lat", "lon", "item_id", "week_num"]
df_rest = np.array([data[:-1] for data in data_points])
mean = np.mean(df_p)
stdev = np.std(df_p)
df_price = (df_p - mean) / stdev

train_size = len(df_rest) - test_size
X = torch.tensor(df_rest, dtype=torch.float)
y = torch.tensor(df_price, dtype=torch.float)
X_train = X[:train_size]
X_test = X[train_size:]
y_train = y[:train_size]
y_test = y[train_size: ]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch.Size([63710, 4]) torch.Size([63710])
torch.Size([250, 4]) torch.Size([250])


In [11]:
class SVM(nn.Module):
    def __init__(self, input_size, output_size, hidden_size=1):
        super(SVM, self).__init__()
        self.gnu = nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=1, batch_first=True)
        self.first = nn.Linear(hidden_size, output_size)
    def forward(self, x): 
        x, _ = self.gnu(x)
        x = self.first(x)
        return x

In [12]:
model = SVM(4, 1, 100)
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=8)

n_epochs = 3000
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch)
        #y_pred = y_pred.unsqueeze(0)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Validation
    if epoch % 100 != 0:
        continue
    model.eval()
    with torch.no_grad():
        y_pred = model(X_train)
        #y_pred = y_pred.unsqueeze(0)
        train_rmse = np.sqrt(loss_fn(y_pred, y_train))
        y_pred = model(X_test)
        #y_batch = y_batch.unsqueeze(0)
        test_rmse = np.sqrt(loss_fn(y_pred, y_test))
    print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch, train_rmse, test_rmse))

/usr/local/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([6])) that is different to the input size (torch.Size([6, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([63710])) that is different to the input size (torch.Size([63710, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(inpu

Epoch 0: train RMSE 0.9913, test RMSE 2.4155
Epoch 100: train RMSE 0.9946, test RMSE 2.3032
Epoch 200: train RMSE 0.9909, test RMSE 2.4052
Epoch 300: train RMSE 1.0030, test RMSE 2.2345
Epoch 400: train RMSE 0.9958, test RMSE 2.2900
Epoch 500: train RMSE 0.9922, test RMSE 2.3369
Epoch 600: train RMSE 0.9924, test RMSE 2.3340
Epoch 700: train RMSE 1.0004, test RMSE 2.5294
Epoch 800: train RMSE 0.9909, test RMSE 2.4101


KeyboardInterrupt: 

In [ ]:
target_mean = X_train.mean()
target_stdev = X_train.std()
x = torch.tensor([[   0., 1006.,  22.],[   4., 1004.,  500.]])
def predict(data_loader):
    model.eval()
    with torch.no_grad():
        for X in data_loader:
            print(X)
            y_star = model(X)
            return y_star * stdev + mean
print(predict(DataLoader(x, batch_size=2, shuffle=False)))